Importing Libraries

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import matplotlib.pyplot as plt

Importing the Training Data

In [ ]:
train_base_dir = 'train'

classes = ['fire', 'nofire']

fire_file_paths = []
nofire_file_paths = []

for class_name in classes:
    class_dir = os.path.join(train_base_dir, class_name)
    
    if class_name == 'fire':
        fire_subclasses = ['Both_smoke_and_fire', 'Smoke_from_fires']
        for subclass_name in fire_subclasses:
            subclass_dir = os.path.join(class_dir, subclass_name)
            file_paths = [os.path.join(subclass_dir, file) for file in os.listdir(subclass_dir)]
            fire_file_paths.extend(file_paths)
    
    elif class_name == 'nofire':
        nofire_subclasses = ['Smoke_confounding_elements', 'Forested_areas_without_confounding_elements', 'Fire_confounding_elements']
        for subclass_name in nofire_subclasses:
            subclass_dir = os.path.join(class_dir, subclass_name)
            file_paths = [os.path.join(subclass_dir, file) for file in os.listdir(subclass_dir)]
            nofire_file_paths.extend(file_paths)

Importing the Testing Data

In [ ]:
test_base_dir = 'test'

classes = ['fire', 'nofire']

fire_test_file_paths = []
nofire_test_file_paths = []

for class_name in classes:
    class_dir = os.path.join(test_base_dir, class_name)
    
    if class_name == 'fire':
        fire_subclasses = ['Both_smoke_and_fire', 'Smoke_from_fires']
        for subclass_name in fire_subclasses:
            subclass_dir = os.path.join(class_dir, subclass_name)
            file_paths = [os.path.join(subclass_dir, file) for file in os.listdir(subclass_dir)]
            fire_test_file_paths.extend(file_paths)
    
    elif class_name == 'nofire':
        nofire_subclasses = ['Smoke_confounding_elements', 'Forested_areas_without_confounding_elements', 'Fire_confounding_elements']
        for subclass_name in nofire_subclasses:
            subclass_dir = os.path.join(class_dir, subclass_name)
            file_paths = [os.path.join(subclass_dir, file) for file in os.listdir(subclass_dir)]
            nofire_test_file_paths.extend(file_paths)

Importing the Validation Data

In [ ]:
val_base_dir = 'val'

classes = ['fire', 'nofire']

fire_test_file_paths = []
nofire_test_file_paths = []

for class_name in classes:
    class_dir = os.path.join(val_base_dir, class_name)
    
    if class_name == 'fire':
        fire_subclasses = ['Both_smoke_and_fire', 'Smoke_from_fires']
        for subclass_name in fire_subclasses:
            subclass_dir = os.path.join(class_dir, subclass_name)
            file_paths = [os.path.join(subclass_dir, file) for file in os.listdir(subclass_dir)]
            fire_test_file_paths.extend(file_paths)
    
    elif class_name == 'nofire':
        nofire_subclasses = ['Smoke_confounding_elements', 'Forested_areas_without_confounding_elements', 'Fire_confounding_elements']
        for subclass_name in nofire_subclasses:
            subclass_dir = os.path.join(class_dir, subclass_name)
            file_paths = [os.path.join(subclass_dir, file) for file in os.listdir(subclass_dir)]
            nofire_test_file_paths.extend(file_paths)

Creating the CNN

In [ ]:
# Define the CNN model
model = keras.Sequential([
    # Convolutional layer 1
    Conv2D(64, (3, 3), activation='relu', input_shape=(128, 128, 3)),
    MaxPooling2D((2, 2)),
    
    # Convolutional layer 2
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    
    # Convolutional layer 3
    Conv2D(256, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    
    # Convolutional layer 4
    Conv2D(512, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    
    # Flatten the output for the fully connected layers
    Flatten(),
    
    # Fully connected layers
    Dense(512, activation='relu'),
    Dropout(0.5),  # Add dropout for regularization
    Dense(256, activation='relu'),
    Dropout(0.3),  # Add dropout for regularization
    Dense(1, activation='sigmoid')  # Output layer for binary classification
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# You can print a summary of the model's architecture
model.summary()

Image Generator

In [ ]:
data_gen = ImageDataGenerator(
    rescale=1.0 / 255.0,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True
)

train_generator = data_gen.flow_from_directory(
    'train',
    target_size=(128, 128),
    batch_size=32,
    class_mode='binary'
)

val_data_gen = ImageDataGenerator(rescale=1.0 / 255.0)

val_generator = data_gen.flow_from_directory(
    'val',
    target_size=(128, 128),
    batch_size=32,
    class_mode='binary',
    shuffle=False
)

Compile and Train Model

In [ ]:
# Train the model on the training data
history = model.fit(train_generator, epochs=3, validation_data=val_generator)  # Adjust the number of epochs as needed

# Save the trained model to a file
model.save('complexCNN.h5')  # Replace 'your_trained_model.h5' with your desired file name

Testing the Model

In [ ]:
# Load the saved model
model = keras.models.load_model('complexCNN.h5')  # Replace with the path to your saved model

# Define data augmentation parameters for testing (you can customize as needed)
test_data_gen = ImageDataGenerator(rescale=1.0 / 255.0)

# Create a data generator for testing
test_generator = test_data_gen.flow_from_directory(
    'test',  # Path to the 'test' directory
    target_size=(128, 128),  # Input image dimensions
    batch_size=32,
    class_mode='binary',  # Binary classification (fire/nofire)
    shuffle=False  # Don't shuffle the data for testing
)

# Evaluate the model on the testing data
test_pred = model.predict(test_generator)
test_labels = test_generator.classes

# Convert probabilities to binary predictions (0 or 1)
test_predictions = (test_pred > 0.5).astype(int)

# Calculate accuracy, precision, recall, and F1 score
test_accuracy = accuracy_score(test_labels, test_predictions)
test_precision = precision_score(test_labels, test_predictions)
test_recall = recall_score(test_labels, test_predictions)
test_f1_score = f1_score(test_labels, test_predictions)

print(f'Test accuracy: {test_accuracy * 100:.2f}%')
print(f'Test precision: {test_precision:.2f}')
print(f'Test recall: {test_recall:.2f}')
print(f'Test F1 Score: {test_f1_score:.2f}')

Plotting Training and Validation Loss

In [ ]:
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.title('Training and Validation Loss')
plt.show()

Plotting Accuracy

In [ ]:
plt.subplot(1, 2, 2)
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.title('Training and Validation Accuracy')
plt.tight_layout()
plt.show()